<a href="https://colab.research.google.com/github/Veronika2311/homeworks_3rd_year/blob/main/%D0%90%D0%B2%D1%82%D0%BE%D0%B1%D1%80%D0%B5%D1%8F_hw3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gensim, Mallet, TF-IDF

In [45]:
!pip install nltk

In [46]:
!pip install spacy

In [47]:
!pip  install pyLDAvis

In [48]:
import nltk; nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [49]:
!python -m spacy download en

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [50]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [51]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [52]:
df = pd.read_json('https://raw.githubusercontent.com/selva86/datasets/master/newsgroups.json')
print(df.target_names.unique())
df.head()

['rec.autos' 'comp.sys.mac.hardware' 'comp.graphics' 'sci.space'
 'talk.politics.guns' 'sci.med' 'comp.sys.ibm.pc.hardware'
 'comp.os.ms-windows.misc' 'rec.motorcycles' 'talk.religion.misc'
 'misc.forsale' 'alt.atheism' 'sci.electronics' 'comp.windows.x'
 'rec.sport.hockey' 'rec.sport.baseball' 'soc.religion.christian'
 'talk.politics.mideast' 'talk.politics.misc' 'sci.crypt']


,content,target,target_names
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,7,rec.autos
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,4,comp.sys.mac.hardware
2,From: twillis@ec.ecn.purdue.edu (Thomas E Will...,4,comp.sys.mac.hardware
3,From: jgreen@amber (Joe Green)\nSubject: Re: W...,1,comp.graphics
4,From: jcm@head-cfa.harvard.edu (Jonathan McDow...,14,sci.space


In [53]:
# Convert to list
data = df.content.values.tolist()


#YOURCODE

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:1])

['From: (wheres my thing) Subject: WHAT car is this!? Nntp-Posting-Host: '
 'rac3.wam.umd.edu Organization: University of Maryland, College Park Lines: '
 '15 I was wondering if anyone out there could enlighten me on this car I saw '
 'the other day. It was a 2-door sports car, looked to be from the late 60s/ '
 'early 70s. It was called a Bricklin. The doors were really small. In '
 'addition, the front bumper was separate from the rest of the body. This is '
 'all I know. If anyone can tellme a model name, engine specs, years of '
 'production, where this car is made, history, or whatever info you have on '
 'this funky looking car, please e-mail. Thanks, - IL ---- brought to you by '
 'your neighborhood Lerxst ---- ']


In [54]:
#tikenize

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp', 'posting', 'host', 'rac', 'wam', 'umd', 'edu', 'organization', 'university', 'of', 'maryland', 'college', 'park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front', 'bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']]


In [ ]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [ ]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [ ]:
!python -m spacy download en_core_web_sm

In [ ]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

In [ ]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

Создайте функцию или серию функций, через которую будет удобно подобрать оптимальное число групп - 1 балл за нахождение оптимального числа групп, 1 балл - если этобудет не руками, а через функцию

Как вы знаете, gensim считает, что каждый текст содержит несколько топиков, но на следвующем этапе вам будет надо создать функцию, которая будет для каждого текста определять один широкий топик, самый главный.

Мв предлагаем сделать это так: создайте счётчик, и каждый раз, когда в тексте будет встречаться одно из слов, соответсвующих данной теме, добавляйте к счетчику его вес. 2 балла

После того, как у вас получится какое-то количество групп (наборов текстов с общим топиком). Внутри каждой из этих групп посчитайте тф_идф для каждого текста (Т. е. возьмите все тексты с одинаковой темой за ваш корпус и посчитайте для каждого из этих текстов тф_идф). 3 балла

Каждому тексту определите слова с пятью самыми высокими тф_идф и запишите их в таблицу

вывод - excel, csv или pandas таблица с текстом, его широким топиком по дженсиму и 5 тф_идф словами для этого текста - 1 балл

Логичность и красота кода - 1 балл

ещё один бонус балл: описать как работает coherence score (не успели обсудить на паре) на русском - словами

In [ ]:
import tqdm

In [ ]:
def choose_score(start, number, rate, corpus, id2word, data_lemmatized):
    #os.environ['MALLET_HOME'] = 'D:/mallet-2.0.8/mallet-2.0.8'
    #mallet_path = 'D:/mallet-2.0.8/mallet-2.0.8/bin/mallet'
    best_coh = 0
    best_num = 0
    best_top = ''
    for i in range(start, number, rate):
        print(i)
        lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=i, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
        # Show Topics
        

        # Compute Coherence Score
        coherence_lda_model = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
        coherence_lda = coherence_lda_model.get_coherence()
        #print('\nCoherence Score: ', coherence_ldamallet)
        if coherence_lda > best_coh:
            best_coh = coherence_lda
            best_num = i
            best_model = lda_model
            print(best_coh, best_num)

    return (best_coh, best_num, best_model)

In [ ]:
choose_score(5, 30, 5, corpus, id2word, data_lemmatized)

In [ ]:
#счётчики для тем

In [ ]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [ ]:
doc_lda = lda_model[corpus]

In [ ]:
 t = lda_model.print_topics()

In [ ]:
list(t)

In [ ]:
topic_dict = {}
for topic in list(t):
    topic = tuple(topic)
    new_top = {}
    for word in topic[1].split(' + '):
        word = word.split('*')
        new_top[word[1].strip('\"\"')] = float(word[0])
    topic_dict[topic[0]] = new_top


In [ ]:
topic_dict[0]

In [ ]:
text_topics = []
for text in data_lemmatized:
    count = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    for word in text:
        for topic, lemmas in topic_dict.items():
            for lemm in lemmas:
                if lemm == word:
                    count[topic] += lemmas[lemm]
    text_topics.append(count.index(max(count)))

In [ ]:
len(text_topics)

In [ ]:
len(data_lemmatized)

# TF-IDF

In [ ]:
import pandas as pd

In [ ]:
df_prefinal = pd.DataFrame()
df_prefinal['texts'] = df['content']
df_prefinal['lemmas'] = data_lemmatized
df_prefinal['topics'] = text_topics
df_prefinal['ind'] = [x for x in range(0,11314)]


In [ ]:
df_prefinal[:10]

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

In [ ]:
df_1 = df_prefinal[df_prefinal['topics'] == 0]

In [ ]:
df_1

In [ ]:
df_1['lemmas'].tolist()[0]

In [ ]:
import numpy as np

def findwords(vect, words):
    sorted_w = np.argsort(vect.data)[:-6:-1]
    return words[vect.indicles[sorted_w]]

In [ ]:
all_topics = {}
for i in range(10):
    list_text = []
    dict_of_topic = {}
    df_1 = df_prefinal[df_prefinal['topics'] == i]
    lemmas = df_1['lemmas'].tolist()
    inds = df_1['ind'].tolist()
    for num, t in enumerate(inds):
        dict_of_topic[t] = lemmas[num]
        list_text.append(' '.join(lemmas[num]))
    vectors = vectorizer.fit_transform(list_text)
    feature_names = np.array(vectorizer.get_feature_names())
    #dense = vectors.todense()
    #denselist = dense.tolist()
    #df = pd.DataFrame(denselist, columns=feature_names)
    3#for text in df.index.tolist():
        #a = df[df[index] == text].tolist

    for index_lemm, l in enumerate(lemmas):
        #tfidf_dict = {}
        text_vect = vectors[index_lemm, :]
        top_words = findwords(text_vect, feature_names)
        all_topics[inds[inedx_lemm]] = top_words


In [ ]:
for i in range(10):
    list_texts = 
    vectors = vectorizer.fit_transform([documentA, documentB, documentC, documentD])
    feature_names = vectorizer.get_feature_names()
    dense = vectors.todense()
    denselist = dense.tolist()
    df = pd.DataFrame(denselist, columns=feature_names)